In [2]:
import pandas as pd
import math
import requests
import re

In [3]:
!ls


 analise_exploratoria.ipynb   data.csv		 teams_and_leagues.csv
 bit_vector.csv		      dumb_stuff.ipynb	 TESTE
'data_>60_pra>80.csv'	      FIFA_HTML
 data_com_dummy.csv	      group_club.ipynb


In [4]:
times_liga = pd.read_csv('teams_and_leagues.csv')
times_liga.head(2)

,url,league_name
0,241,Spain Primera Division
1,11,English Premier League


### Para baixar os html's dos times

In [5]:
url_1 = "https://sofifa.com/team/"
url_2 = "/?r=190071&set=true"
team_name = []
!mkdir FIFA_HTML

for index,row in times_liga.iterrows():
    url_3 = url_1 + str(row['url']) + url_2
    print(row['url'])
    print(url_3)
    path = "FIFA_HTML/"+str(row['url'])+".txt"
    f=open(path, "w")
    html_team_txt = str(requests.get(url_3).text)
    f.write(html_team_txt)
    

mkdir: cannot create directory ‘FIFA_HTML’: File exists
241
https://sofifa.com/team/241/?r=190071&set=true
11
https://sofifa.com/team/11/?r=190071&set=true
5
https://sofifa.com/team/5/?r=190071&set=true
243
https://sofifa.com/team/243/?r=190071&set=true
1
https://sofifa.com/team/1/?r=190071&set=true
9
https://sofifa.com/team/9/?r=190071&set=true
22
https://sofifa.com/team/22/?r=190071&set=true
21
https://sofifa.com/team/21/?r=190071&set=true


KeyboardInterrupt: 

### Para extrair os dados 

In [9]:
team_name = []
team_def_sty = []
team_atk_sty = []
team_dom_prestige = []
team_int_prestige = []

for index,row in times_liga.iterrows():
    path = "FIFA_HTML/"+str(row['url'])
    f=open(path, "r")
    
    html_team_txt = str(f.read())
    start_team = '<title>'
    end_team = ' FIFA'
    
    start_def_sty = 'Defensive Style'
    end_def_sty = '</span>'
    
    start_atk_sty = 'Offensive Style'
    end_atk_sty = '</span>' 
    
    team = html_team_txt.split(start_team)[1].split(end_team)[0]
    def_sty = ""
    atk_sty = ""
   
    dom_prestige = ""
    int_prestige = ""

    start_int_prest =  "International Prestige"
    end_int_prest = "</span>"
    
    start_dom_prest =  "Domestic Prestige"
    end_dom_prest = "</span>"
    
    
    if("Defensive Style" in html_team_txt):
        def_sty = html_team_txt.split(start_def_sty)[1].split(end_def_sty)[0]
        i = def_sty.index(""".">""")
        def_sty = def_sty[i+3:]
    
    if("Offensive Style" in html_team_txt):
        atk_sty = html_team_txt.split(start_atk_sty)[1].split(end_atk_sty)[0]     
        i = atk_sty.index(""".">""")
        atk_sty = atk_sty[i+3:]
    
    if(start_int_prest in html_team_txt):
        int_prestige = html_team_txt.split(start_int_prest)[1].split(end_int_prest)[0]  
        i = int_prestige.index("""p">""")
        int_prestige = int_prestige[i+3:]

        
    if(start_dom_prest in html_team_txt):
        dom_prestige = html_team_txt.split(start_dom_prest)[1].split(end_dom_prest)[0]     
        i = dom_prestige.index("""p">""")
        dom_prestige = dom_prestige[i+3:]
    
    team_dom_prestige.append(dom_prestige)
    team_int_prestige.append(int_prestige)
    team_name.append(team)
    team_def_sty.append(def_sty)
    team_atk_sty.append(atk_sty)
    


In [10]:
times_liga['Club'] = team_name
times_liga['Defensive Style'] = team_def_sty
times_liga['Offensive Style'] = team_atk_sty
times_liga['Domestic Prestige'] = team_dom_prestige
times_liga['International Prestige'] = team_int_prestige

In [12]:
times_liga.sample(5)

,url,league_name,Club,Defensive Style,Offensive Style,Domestic Prestige,International Prestige
485,420,Polish T-Mobile Ekstraklasa,G\xc3\xb3rnik Zabrze,Pressure On Heavy Touch,Long Ball,5,2
977,1971,Holland Eredivisie,Excelsior,Drop Back,Fast Build Up,2,1
854,111810,Spanish Segunda División,Club Polideportivo Granada 74,,,5,1
941,110106,Russian Premier League,Mordovia Saransk,,,5,4
499,101148,Japanese J. League Division 1,Cerezo Osaka,Balanced,Balanced,7,5


In [13]:
times_liga.to_csv('club.csv', encoding='utf-8')


#### Agrupamento dos jogadores por clubes

In [15]:
df = pd.read_csv("data.csv")

In [17]:
df.head(1)

,Unnamed: 0,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,Argentina,94,94,FC Barcelona,110500000,565000,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0


In [21]:
df_club = df.groupby("Club").mean()
df_club = df_club.round(0)

In [137]:
df_club.columns

Index(['Unnamed: 0', 'ID', 'Age', 'Overall', 'Potential', 'Value', 'Wage',
       'Special', 'International Reputation', 'Weak Foot', 'Skill Moves',
       'Jersey Number', 'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF',
       'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB',
       'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing',
       'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes', 'Release Clause'],
      dtype='object')

In [23]:
df_club.loc['FC Barcelona']

Unnamed: 0            3293.0
ID                  210877.0
Age                     24.0
Overall                 78.0
Potential               85.0
                     ...    
GKHandling              18.0
GKKicking               19.0
GKPositioning           19.0
GKReflexes              19.0
Release Clause    54939394.0
Name: FC Barcelona, Length: 75, dtype: float64